In [1]:
!pip install -q transformers
!pip install tqdm

     |████████████████████████████████| 4.4 MB 9.9 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
tqdm.pandas()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
politifact = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/politifact.csv")
gossipcop = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/gossipcop.csv")
kaggle = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/kaggle_kdd_features.csv")
liar = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/liar_features.csv")
coaid = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/coaid_features.csv")
celebrity = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/celebrity.csv")
fakenews = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/fakenews.csv")

In [6]:
sentiment_pipeline = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest",truncation=True, max_length=512)

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [7]:
def sentiment_score(x):
  try:
    ans = sentiment_pipeline(x)
    return (ans[0]['label'],ans[0]['score'])
  except Exception as e:
    print(e)
    pass

def sentiment_count(df):
  df = df.dropna(subset=["text"])
  df["sentiment"]  = df["text"].progress_apply(lambda x: sentiment_score(x)) 
  df["sentiment_label"] = df["sentiment"].apply(lambda x: x[0])
  df["sentiment_score"] = df["sentiment"].apply(lambda x: x[1])
  return df, df.groupby(["label","sentiment_label"]).size().reset_index(name="counts")

## Politifact

In [ ]:
new_politifact, politifact_groupby = sentiment_count(politifact)

In [ ]:
new_politifact.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/politifact_sentiment.csv",index=False)
politifact_groupby

## Kaggle

In [ ]:
new_kaggle, kaggle_groupby = sentiment_count(kaggle)

In [ ]:
new_kaggle.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/kaggle_sentiment.csv",index=False)
kaggle_groupby

## Liar

In [ ]:
new_liar, liar_groupby = sentiment_count(liar)

In [ ]:
new_liar.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/liar_sentiment.csv",index=False)
liar_groupby

## Coaid

In [ ]:
new_coaid, coaid_groupby = sentiment_count(coaid)

In [ ]:
new_coaid.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/coaid_sentiment.csv",index=False)
coaid_groupby

## Celebrity

In [ ]:
new_celebrity, celebrity_groupby = sentiment_count(celebrity)

In [ ]:
new_celebrity.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/celebrity_sentiment.csv",index=False)
celebrity_groupby

## Fake News

In [ ]:
new_fakenews, fakenews_groupby = sentiment_count(fakenews)

In [ ]:
new_fakenews.to_csv("/content/drive/MyDrive/hate-fake-online-asonam/fakenews_sentiment.csv",index=False)
fakenews_groupby

## Gossipcop

In [9]:
new_gossipcop, gossipcop_groupby = sentiment_count(gossipcop)

100%|██████████| 19711/19711 [6:49:05<00:00,  1.25s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [11]:
new_gossipcop.to_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/gossipcop_sentiment.csv",index=False)
gossipcop_groupby

,label,sentiment_label,counts
0,fake,Negative,512
1,fake,Neutral,3372
2,fake,Positive,870
3,real,Negative,812
4,real,Neutral,9189
5,real,Positive,4956
